<h1 style="color:DodgerBlue">Индивидальный проект</h1>

<h2 style="color:DodgerBlue">Название проекта:</h2>

----

### Вариант задания N6


<h2 style="color:DodgerBlue">Описание проекта:</h2>

----

Создать базовый класс Movie в C#, который будет представлять информацию о
фильмах. На основе этого класса разработать 2-3 производных класса,
демонстрирующих принципы наследования и полиморфизма. В каждом из классов
должны быть реализованы новые атрибуты и методы, а также переопределены
некоторые методы базового класса для демонстрации полиморфизма.

#### Дополнительное задание
Добавьте к сущестующим классам (базовыму и производным 3-4 атрибута и метода) и реализуйте простое, сложное и множественное наследование

<h2 style="color:DodgerBlue">Реализация:</h2>

----

In [2]:
public interface IStreamable
{
    bool IsAvailableForStreaming { get; set; }
    void Stream();
    string GetStreamingInfo();
}
public class Movie
{
    private string _title;
    private string _director;
    private int _releaseYear;
    private double _rating;
    private int _duration;
    private string _country;
    private List<string> _actors;

    public Movie(string title, string director, int releaseYear, int duration, string country)
    {
        Title = title;
        Director = director;
        ReleaseYear = releaseYear;
        Duration = duration;
        Country = country;
        _rating = 0;
        _actors = new List<string>();
    }
    public string Title
    {
        get => _title;
        set => _title = !string.IsNullOrWhiteSpace(value) ? value : "Неизвестный фильм";
    }

    public string Director
    {
        get => _director;
        set => _director = !string.IsNullOrWhiteSpace(value) ? value : "Неизвестный режиссер";
    }

    public int ReleaseYear
    {
        get => _releaseYear;
        set => _releaseYear = value >= 1888 ? value : 1888;
    }

    public int Duration
    {
        get => _duration;
        set => _duration = value > 0 ? value : 60;
    }

    public string Country
    {
        get => _country;
        set => _country = !string.IsNullOrWhiteSpace(value) ? value : "Неизвестно";
    }

    public double Rating
    {
        get => _rating;
        private set => _rating = Math.Clamp(value, 0, 10);
    }

    public List<string> Actors => new List<string>(_actors);
    public virtual void AddActor(string actor)
    {
        if (!string.IsNullOrWhiteSpace(actor) && !_actors.Contains(actor))
        {
            _actors.Add(actor);
        }
    }

    public void AddActors(params string[] actors)
    {
        foreach (var actor in actors)
        {
            AddActor(actor);
        }
    }

    public string GetDurationInfo()
    {
        int hours = Duration / 60;
        int minutes = Duration % 60;
        return $"{hours}ч {minutes}мин";
    }

    public int GetMovieAge()
    {
        return DateTime.Now.Year - ReleaseYear;
    }

    public bool IsClassic()
    {
        return GetMovieAge() >= 25;
    }

    public virtual string GetInfo()
    {
        return $"Фильм: {Title}\nРежиссер: {Director}\nГод: {ReleaseYear} ({GetMovieAge()} лет назад)";
    }

    public virtual void Watch()
    {
        Console.WriteLine($"Смотрим: {Title}");
    }

    public virtual void Rate(double score)
    {
        Rating = score;
        Console.WriteLine($"Оценка '{Title}': {Rating:F1}/10");
    }

    public void DisplayCast()
    {
        if (_actors.Count > 0)
        {
            Console.WriteLine($"Актеры ({_actors.Count}): {string.Join(", ", _actors)}");
        }
    }
}

public class Documentary : Movie
{
    private string _theme;
    private bool _isEducational;

    public Documentary(string title, string director, int releaseYear, int duration, string country, string theme, bool isEducational)
        : base(title, director, releaseYear, duration, country)
    {
        Theme = theme;
        IsEducational = isEducational;
    }

    public string Theme
    {
        get => _theme;
        set => _theme = !string.IsNullOrWhiteSpace(value) ? value : "Общая тематика";
    }

    public bool IsEducational
    {
        get => _isEducational;
        set => _isEducational = value;
    }

    public void DisplayEducationalValue()
    {
        Console.WriteLine(IsEducational ? 
            "Этот документальный фильм имеет образовательную ценность" :
            "Этот документальный фильм предназначен для развлечения");
    }

    public string GetThemeDescription()
    {
        return $"Тематика: {Theme}";
    }

    public override void Watch()
    {
        base.Watch();
        Console.WriteLine($"Тема: {Theme}");
        DisplayEducationalValue();
    }

    public override string GetInfo()
    {
        return base.GetInfo() + $"\nТип: Документальный\n{GetThemeDescription()}";
    }
}

public class FeatureFilm : Movie, IStreamable
{
    private string _genre;
    private decimal _budget;

    public FeatureFilm(string title, string director, int releaseYear, int duration, string country, string genre, decimal budget)
        : base(title, director, releaseYear, duration, country)
    {
        Genre = genre;
        Budget = budget;
        IsAvailableForStreaming = false;
    }

    public string Genre
    {
        get => _genre;
        set => _genre = !string.IsNullOrWhiteSpace(value) ? value : "Драма";
    }

    public decimal Budget
    {
        get => _budget;
        set => _budget = value >= 0 ? value : 0;
    }

    public bool IsAvailableForStreaming { get; set; }
    public decimal EstimateBoxOffice()
    {
        double multiplier = Rating >= 8 ? 5.0 : Rating >= 6 ? 3.0 : 1.5;
        return Budget * (decimal)multiplier;
    }

    public string GetGenreInfo()
    {
        return $"Жанр '{Genre}' - {GetGenreDescription()}";
    }

    private string GetGenreDescription()
    {
        return Genre.ToLower() switch
        {
            "комедия" => "юмор и развлечения",
            "драма" => "эмоциональная глубина",
            "боевик" => "экшн и приключения",
            "фэнтези" => "магия и воображение",
            _ => "различные темы"
        };
    }

    public override void Rate(double score)
    {
        double adjustedScore = score * GetGenreCoefficient();
        base.Rate(adjustedScore);
        Console.WriteLine($"С учетом жанра '{Genre}' (коэффициент: {GetGenreCoefficient():F2})");
    }

    private double GetGenreCoefficient()
    {
        return Genre.ToLower() switch
        {
            "комедия" => 0.9,
            "драма" => 1.1,
            "боевик" => 0.95,
            _ => 1.0
        };
    }

    public override string GetInfo()
    {
        return base.GetInfo() + $"\nТип: Игровой\n{GetGenreInfo()}\nБюджет: ${Budget:N0}";
    }

    public void Stream()
    {
        if (IsAvailableForStreaming)
        {
            Console.WriteLine($"Стримим '{Title}' онлайн");
        }
        else
        {
            Console.WriteLine($"'{Title}' недоступен для стриминга");
        }
    }

    public string GetStreamingInfo()
    {
        return IsAvailableForStreaming ? 
            $"Фильм '{Title}' доступен для онлайн-просмотра" :
            $"Фильм '{Title}' недоступен для стриминга";
    }
}

public class AnimatedMovie : Movie
{
    private string _animationStudio;
    private string _targetAudience;

    public AnimatedMovie(string title, string director, int releaseYear, int duration, string country, string animationStudio, string targetAudience)
        : base(title, director, releaseYear, duration, country)
    {
        AnimationStudio = animationStudio;
        TargetAudience = targetAudience;
    }

    public string AnimationStudio
    {
        get => _animationStudio;
        set => _animationStudio = !string.IsNullOrWhiteSpace(value) ? value : "Неизвестная студия";
    }

    public string TargetAudience
    {
        get => _targetAudience;
        set => _targetAudience = !string.IsNullOrWhiteSpace(value) ? value : "Для всей семьи";
    }

    public void DisplayStudioInfo()
    {
        Console.WriteLine($"Анимационная студия: {AnimationStudio}");
    }

    public bool IsForChildren()
    {
        return TargetAudience.ToLower().Contains("дети") || 
               TargetAudience.ToLower().Contains("семья");
    }

    public override void Watch()
    {
        Console.WriteLine($"Смотрим анимацию: {Title}");
        Console.WriteLine($"Студия: {AnimationStudio}");
        Console.WriteLine(IsForChildren() ? "Отлично подходит для детей!" : "Рекомендуется для взрослой аудитории");
    }

    public override string GetInfo()
    {
        return base.GetInfo() + $"\nТип: Анимационный\nСтудия: {AnimationStudio}\nАудитория: {TargetAudience}";
    }

    public override void AddActor(string actor)
    {
        base.AddActor(actor);
        Console.WriteLine($"(Озвучка: {actor})");
    }
}
        Console.WriteLine("=== СИСТЕМА УПРАВЛЕНИЯ ФИЛЬМАМИ ===\n");
        var documentary = new Documentary("Планета Земля", "Дэвид Аттенборо", 2006, 180, "Великобритания", "Природа и животные", true);
        var featureFilm = new FeatureFilm("Крестный отец", "Фрэнсис Коппола", 1972, 175, "США", "Драма", 6000000m);
        var animatedMovie = new AnimatedMovie("Король Лев", "Роджер Аллерс", 1994, 89, "США", "Disney", "Для всей семьи");
        featureFilm.AddActors("Марлон Брандо", "Аль Пачино", "Джеймс Каан");
        animatedMovie.AddActor("Мэттью Бродерик");
        documentary.AddActor("Дэвид Аттенборо");
        featureFilm.IsAvailableForStreaming = true;

        Movie[] movies = { documentary, featureFilm, animatedMovie };

        foreach (var movie in movies)
        {
            Console.WriteLine("====================================");
            Console.WriteLine(movie.GetInfo());
            Console.WriteLine("------------------------------------");
            movie.DisplayCast();
            Console.WriteLine($"Продолжительность: {movie.GetDurationInfo()}");
            Console.WriteLine($"Классика: {(movie.IsClassic() ? "Да" : "Нет")}");
            
            movie.Watch();
            
            Random rnd = new Random();
            double rating = rnd.NextDouble() * 5 + 5; 
            movie.Rate(rating);
            
            if (movie is Documentary doc)
            {
                doc.DisplayEducationalValue();
            }
            else if (movie is FeatureFilm film)
            {
                Console.WriteLine($"Прогнозируемые сборы: ${film.EstimateBoxOffice():N0}");
                film.Stream();
            }
            else if (movie is AnimatedMovie anim)
            {
                anim.DisplayStudioInfo();
                Console.WriteLine($"Для детей: {(anim.IsForChildren() ? "Да" : "Нет")}");
            }
            
            Console.WriteLine();
        }

=== СИСТЕМА УПРАВЛЕНИЯ ФИЛЬМАМИ ===

(Озвучка: Мэттью Бродерик)
Фильм: Планета Земля
Режиссер: Дэвид Аттенборо
Год: 2006 (19 лет назад)
Тип: Документальный
Тематика: Природа и животные
------------------------------------
Актеры (1): Дэвид Аттенборо
Продолжительность: 3ч 0мин
Классика: Нет
Смотрим: Планета Земля
Тема: Природа и животные
Этот документальный фильм имеет образовательную ценность
Оценка 'Планета Земля': 6.7/10
Этот документальный фильм имеет образовательную ценность

Фильм: Крестный отец
Режиссер: Фрэнсис Коппола
Год: 1972 (53 лет назад)
Тип: Игровой
Жанр 'Драма' - эмоциональная глубина
Бюджет: $6,000,000
------------------------------------
Актеры (3): Марлон Брандо, Аль Пачино, Джеймс Каан
Продолжительность: 2ч 55мин
Классика: Да
Смотрим: Крестный отец
Оценка 'Крестный отец': 10.0/10
С учетом жанра 'Драма' (коэффициент: 1.10)
Прогнозируемые сборы: $30,000,000
Стримим 'Крестный отец' онлайн

Фильм: Король Лев
Режиссер: Роджер Аллерс
Год: 1994 (31 лет назад)
Тип: Анимацион